In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import bisect
import copy
import logging
import math
from statistics import mean
import pandas as pd
from jqdata import get_all_trade_days
from jqdata import bond
from datetime import datetime, timedelta

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.DEBUG)

# 低于110元的转债我们看作低估
UNDERRATE_PRICE = 110

# 聚宽的可转债数据从2018-09-13开始记录
JQDATA_BEGIN_DATE = '2018-09-13'

        
class ConvertBondBeta(object):
    """得到指定时间的可转债基本信息，包括:
        市场总量
        低估转债(<110元)市场总量
        全部转债价格算术平均值
        全部转债溢价率算术平均值
        
        指定一个时间段，算出当前转债市场指标的历史百分位
    """
    
    def __init__(self, base_date=None, history_days=365*3):
        """
        input:
            base_date: 查询时间，格式为'yyyy-MM-dd'，默认为当天
            history_days: 默认历史区间位前3年
        """
        if not base_date:
            self._base_date = datetime.now().date()
        else:
            self._base_date = datetime.strptime(base_date, '%Y-%m-%d')
            
        self._begin_date = self._base_date - timedelta(history_days)
        self._end_date = self._base_date
        
        if self._begin_date < datetime.strptime(JQDATA_BEGIN_DATE, '%Y-%m-%d'):
            self._begin_date = datetime.strptime(JQDATA_BEGIN_DATE, '%Y-%m-%d')
        
        self._begin_date = self._begin_date.strftime('%Y-%m-%d')
        self._end_date = self._end_date.strftime('%Y-%m-%d')
        self._base_date = self._base_date.strftime('%Y-%m-%d')
        
    def get_bonds(self, date=None):
        """
        获取指定日期的可转债市场正常存续的转债的基本信息
        
            code: 可转债代码
            short_name: 可转债名称
            raise_fund_volume: 发行总量(元)
            current_fund_volume: 当前存量总量(元)
            price: 收盘价格
            day_market_volume: 当日市场成交总价
            convert_premium_ratio: 转股溢价率
            convert_price: 转股价格 (如果没有下修的话就是约定转股价)
            stock_price: 正股价格
            last_cash_date: 最终兑付日(到期时间)
            double_low: 转债价格+溢价率X100
            ytm: 计算方式比较复杂，暂缺失
        """
        
        if date is None:
            date = datetime.strptime(self._base_date, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(date, '%Y-%m-%d').date()
        
        df_bonds = bond.run_query(
                query(bond.CONBOND_BASIC_INFO).filter(
                                    bond.CONBOND_BASIC_INFO.bond_type_id == 703013,
                                    bond.CONBOND_BASIC_INFO.list_status_id.in_(['301001', '301099']),
                                    bond.CONBOND_BASIC_INFO.interest_begin_date < date,
                                    bond.CONBOND_BASIC_INFO.last_cash_date >= date
                                ).order_by('code').limit(10000)
            )        
    
        bond_list = []
        
        total_market = 0.0
        for index, row in df_bonds.iterrows():            
            bond_info = {
                        'code': row['code'], 
                        'short_name': row['short_name'], 
                        'stock_code': row['company_code'],
                        'convert_price': row['convert_price'],
                        'last_cash_date': row['last_cash_date']
            }
            
            if str(row['list_status_id']) == '301099':
                # issue-2: CONBOND_BASIC_INO表中数据更新不及时，需要去BOND_BASIC_INFO中确认一下
                bond_basic_info = bond.run_query(
                                query(bond.BOND_BASIC_INFO).filter(
                                    bond.BOND_BASIC_INFO.code == row['code']
                                )
                              )            
                if str(bond_basic_info['list_status_id'][0]) == '301099':
                    # 未上市
                    continue            
            
            if row['list_date'] and row['list_date'] > date:
                # 只发布了信息，还没有正式上市，暂不记入
                continue
            
            
            # 发行总量(万元)
            if not math.isnan(row['actual_raise_fund']):
                bond_info['raise_fund_volume'] = float(row['actual_raise_fund']) * 10000
            else:
                bond_info['raise_fund_volume'] = float(row['plan_raise_fund']) * 10000
            bond_info['current_fund_volume'] = bond_info['raise_fund_volume']
            
            # 转股信息
            bond_stock = bond.run_query(
                query(bond.CONBOND_DAILY_CONVERT).filter(
                                    bond.CONBOND_DAILY_CONVERT.code == bond_info['code'], 
                                    bond.CONBOND_DAILY_CONVERT.date <= date,
                                )
            )   
            
            # 统计转股信息，如果99%转股，就代表强赎退市，暂不记入，另外要修正存量债券数目
            if not bond_stock['acc_convert_ratio'].empty:
                if bond_stock['acc_convert_ratio'].iloc[-1] >= 99.5:
                    continue
                else:
                    bond_info['current_fund_volume'] = bond_info['raise_fund_volume'] * \
                            (100.0 - bond_stock['acc_convert_ratio'].iloc[-1]) / 100.0
            
            # 先取得转股价，然后取得正股收盘价，然后计算溢价率：转股溢价=（100/转股价格）*正股收盘价-可转债收盘价）
            # 转股价如果有下修，先取得下修转股价
            if not bond_stock['convert_price'].empty:
                bond_info['convert_price'] =  float(bond_stock['convert_price'].iloc[-1])                
            
            # 当前市场收盘价格
            bond_market = bond.run_query(
                query(bond.CONBOND_DAILY_PRICE).filter(
                                    bond.CONBOND_DAILY_PRICE.code == bond_info['code'], 
                                    bond.CONBOND_DAILY_PRICE.date <= date,
                                )
            )
            try:
                bond_info['price'] =  float(bond_market['close'].iloc[-1])
                bond_info['day_market_volume'] = float(bond_market['money'].iloc[-1])
            except Exception:
                # 有部分还没有公布信息的先跳过
                bond_info['price'] =  float(row['par'])
                continue
            
            if bond_info['price'] < 1:
                # 停牌
                continue            

                
            # 获取正股价格
            df_stock_price = get_price(bond_info['stock_code'],
                                       count = 7,
                                       end_date= date,
                                       frequency='daily', 
                                       fields=['close'])
            bond_info['stock_price'] = df_stock_price['close'][-1]
            bond_info['convert_premium_ratio'] = (bond_info['price'] - 100/bond_info['convert_price']*bond_info['stock_price']) / \
                                                 (100/bond_info['convert_price']*bond_info['stock_price'])
            bond_info['double_low'] = bond_info['price'] + bond_info['convert_premium_ratio'] * 100
                      
            bond_list.append(bond_info)
            
        bond_list = sorted(bond_list, key=lambda x: x['double_low'])
        return bond_list
        
            
    def get_bonds_factors(self, bond_list=None):
        """
        获取当前时间的市场总量, 低估转债市场总量，指数平均价格，指数平均溢价率

        output:
             (total_market(元), underrate_market, avg_price, avg_premium_ratio)
        """
        
        if bond_list is None:
            bond_list = self.get_bonds()
            
        total_market = sum([bond['current_fund_volume'] for bond in bond_list])
        underrate_market = sum([bond['current_fund_volume'] for bond in bond_list if bond['price'] <= UNDERRATE_PRICE])
        avg_price = mean([bond['price'] for bond in bond_list])
        avg_premium_ratio = mean([bond['convert_premium_ratio'] for bond in bond_list])
        return (total_market, underrate_market, avg_price, avg_premium_ratio)
                        
        

In [3]:
class StockBeta(object):
    
    def __init__(self, stock_code, index_type=0, base_date=None, history_days=365*5):
        """
        input:
            index_code: 要查询指数的代码
            index_type: 1为等权重方式计算，0为按市值加权计算
            base_date: 查询时间，格式为'yyyy-MM-dd'，默认为当天
            history_days: 默认历史区间位前八年
        """
        self._stock_code = stock_code
        self._index_type = index_type
        if not base_date:
            self._base_date = datetime.now().date() - timedelta(1)
        else:
            self._base_date = datetime.strptime(base_date, '%Y-%m-%d')
            
        self._begin_date = self._base_date - timedelta(history_days)
        self._end_date = self._base_date
        
        self._begin_date = self._begin_date.strftime('%Y-%m-%d')
        self._end_date = self._end_date.strftime('%Y-%m-%d')
        self._base_date = self._base_date.strftime('%Y-%m-%d')
            
    def get_stock_beta_factor(self, day=None):
        """
        获取当前时间的pe, pb值
        
        input:
            day: datetime.date类型，如果为None，默认代表取当前时间

        output:
            (pe, pb, roe)
        """
        if not day:
            day = datetime.strptime(self._base_date, '%Y-%m-%d')
        
        stocks = [self._stock_code]
        q = query(
            valuation.pe_ratio, valuation.pb_ratio, valuation.circulating_market_cap
        ).filter(
            valuation.code.in_(stocks)
        )

        df = get_fundamentals(q, day)

        df = df[df['pe_ratio']>0]

        if len(df)>0:
            if(self._index_type == 0):
                pe = df['circulating_market_cap'].sum() / (df['circulating_market_cap']/df['pe_ratio']).sum()
                pb = df['circulating_market_cap'].sum() / (df['circulating_market_cap']/df['pb_ratio']).sum()
            else:
                pe = df['pe_ratio'].size / (1/df['pe_ratio']).sum()
                pb = df['pb_ratio'].size / (1/df['pb_ratio']).sum()
            return (pe, pb, pb/pe)
        else:
            return (None, None, None)
        
    def get_stock_beta_history_factors(self, interval=7):
        """
        获取任意指数一段时间的历史 pe,pb 估值列表，通过计算当前的估值在历史估值的百分位，来判断当前市场的估值高低。
        由于加权方式可能不同，可能公开的估值数据有差异，但用于判断估值相对高低没有问题

        input：
            interval: 计算指数估值的间隔天数，增加间隔时间可提高计算性能

        output：
            result:  指数历史估值的 DataFrame，index 为时间，列为pe，pb,roe
        """
        all_days = get_all_trade_days()

        pes = []
        roes = []
        pbs = []
        days = []

        begin = datetime.strptime(self._begin_date, '%Y-%m-%d').date()
        end = datetime.strptime(self._end_date, '%Y-%m-%d').date()  
        i = 0
        for day in all_days:
            if(day <= begin or day >= end):
                continue

            i += 1

            if(i % interval != 0):
                continue

            pe, pb, roe = self.get_stock_beta_factor(day)
            if pe and pb and roe:
                pes.append(pe)
                pbs.append(pb)
                roes.append(roe)
                days.append(day)

        result = pd.DataFrame({'pe':pes,'pb':pbs, 'roe':roes}, index=days)
        return result
    
    def get_quantile_of_history_factors(self, factor, history_list):
        """
            获取某个因子在历史上的百分位，比如当前PE处于历史上的70%区间，意味着历史PE有70%都在当前值之下

        input:
            factor: beta因子
            history_list: 历史估值列表, DataFrame

        output:
            quantile: 历史估值百分位 (0.7)
        """
        factors = [history_list.quantile(i / 10.0)  for i in range(11)]    
        idx = bisect.bisect(factors, factor)
        if idx < 10:
            quantile = idx - (factors[idx] - factor) / (factors[idx] - factors[idx-1])   
            return quantile / 10.0    
        else:
            return 1.0


In [4]:
DOUBLE_LOW_VALUE = 130
DOUBLE_LOW_PRICE = 108

class DLowStrategy(object):
    
    # 双低策略转债个数
    EXPECTED_ITEMS_COUNT = 20
    
    def __init__(self, bond_list, base_date):
        self._bond_list = copy.deepcopy(bond_list)
        self._base_date = datetime.strptime(base_date, "%Y-%m-%d").date()
        
        
    def _set_stock_info(self, bond_list):
        """增加正股估值信息
        
            stock_pb: pb
            stock_pe: pe
            stock_pb_quantile: pb百分位
            stock_pe_quantile: pe百分位
        """
        for bond in bond_list:
            stock = StockBeta(bond['stock_code'])
            bond['stock_pe'], bond['stock_pb'], bond['stock_roe'] = stock.get_stock_beta_factor()
            
            if bond['stock_pe'] is None or bond['stock_pb'] is None:
                # 正股报表有问题，剔除
                if bond in self._bond_list:
                    self._bond_list.remove(bond)
                continue
            
            history_factors = stock.get_stock_beta_history_factors()
            bond['stock_pb_quantile'] = stock.get_quantile_of_history_factors(
                                                bond['stock_pb'], history_factors['pb'])
            bond['stock_pe_quantile'] = stock.get_quantile_of_history_factors(
                                                bond['stock_pe'], history_factors['pe'])
            
    def _filter_last_cash_date(self, bond_list):
        """筛选掉最近到期项
        """
        filter_bond_list = filter(
            lambda x: x['last_cash_date'] - self._base_date > timedelta(360),
            bond_list
        )
        return list(filter_bond_list)
        
            
    def _filter_pb_pe_quantile(self, bond_list):
        """筛选pb, pe历史百分位
        """
        self._set_stock_info(bond_list)
        filter_bond_list = filter(
            lambda x: x['stock_pb'] > 1.3 and x['stock_pb_quantile'] < 0.8 and x['stock_pe_quantile'] < 0.8,
            bond_list
        )
        return list(filter_bond_list)
        
    def _filter_pb(self, bond_list):
        """筛选PB>1.3防止下修转股价时破净限制
        """
        self._set_stock_info(bond_list)
        filter_bond_list = filter(
            lambda x: x.get('stock_pb', None) and x['stock_pb'] > 1.3,
            bond_list
        )
        return list(filter_bond_list)

        
        
    def _filter_current_fund_volume(self, bond_list):
        """剩余规模>1亿，且<10亿元的转债
        """
        filter_bond_list = filter(
            lambda x: x['current_fund_volume'] > pow(10, 8) and x['current_fund_volume'] < pow(10, 9),
            bond_list
        )
        return list(filter_bond_list)

    def _filter_current_market_volume(self, bond_list):
        """当日市场成交额>100万
        """
        filter_bond_list = filter(
            lambda x: x['day_market_volume'] > pow(10, 6),
            bond_list
        )
        return list(filter_bond_list)
        
    def _filter_convert_premium_ratio(self, bond_list):
        """溢价率小于15%
        """
        filter_bond_list = filter(
            lambda x: x['convert_premium_ratio'] < 0.15,
            bond_list
        )
        return list(filter_bond_list)
        
        
    def _filter_double_low(self, bond_list):
        """双低小于125-130
        """
        filter_bond_list = filter(
            lambda x: x['double_low'] < DOUBLE_LOW_VALUE,
            bond_list
        )
        return list(filter_bond_list)
    
    def _filter_price(self, bond_list):
        """价格过滤
        """
        filter_bond_list = filter(
            lambda x: x['price'] < DOUBLE_LOW_PRICE,
            bond_list
        )
        return list(filter_bond_list)    
    
    def get_support_bonds(self, filter_pb=False, filter_pb_pe_quantile=False, filter_price=False):
        """
        剩余规模>1亿，且<10亿元的转债
        
        到期时间>一年
        
        当前成交额>100万元的转债
        
        溢价率小于15%,主要是防止市场下跌时杀转债溢价；正股下跌，带动转债价格下跌；溢价率低的转债安全垫更厚；
        另外要注意折价的情况，最典型的就是2020年初的英联转债，折价转债是否值得入手，这个需要仔细研究，
        
        到期税后收益率大于0的转债
        
        取双低值<DOUBLE_LOW_VALUE的转债
        
        筛选PB>1.3防止下修转股价时破净限制
        
        filter_pb_pe_quantile=True, 筛选pb, pe历史百分位
        """
        support_bond_list = self._bond_list
        
        support_bond_list = self._filter_current_fund_volume(support_bond_list)
        support_bond_list = self._filter_current_market_volume(support_bond_list)
        support_bond_list = self._filter_convert_premium_ratio(support_bond_list)
        support_bond_list = self._filter_double_low(support_bond_list)
        support_bond_list = self._filter_last_cash_date(support_bond_list)
        
        
        if filter_pb:
            support_bond_list = self._filter_pb(support_bond_list)
        
        if filter_pb_pe_quantile:
            support_bond_list = self._filter_pb_pe_quantile(support_bond_list)
            
        if filter_price:
            support_bond_list = self._filter_price(support_bond_list)                
            
        return support_bond_list


In [5]:
# 测试
import pandas as pd
from datetime import datetime, timedelta
from jqfactor import *

import warnings

base_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
#base_date = datetime.now()strftime('%Y-%m-%d')
#base_date = '2019-02-24'

index_bond = ConvertBondBeta(base_date=base_date)
print(base_date)
print("=========================")

# 获取指定时间转债列表及统计值 
bond_list = index_bond.get_bonds(base_date)
#pd.DataFrame(bond_list)

2020-12-03


In [6]:
# 先不采用pe, pb百分位筛选
bond_list_a = DLowStrategy(bond_list, base_date).get_support_bonds()
pd.DataFrame(bond_list_a)

,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,price,raise_fund_volume,short_name,stock_code,stock_price
0,113601,0.078055,16.98,5.433100e+08,1.363249e+07,118.785515,2026-08-21,110.980,5.433100e+08,塞力转债,603716.XSHG,17.48
1,123074,-0.078081,29.53,3.245000e+08,4.184119e+09,121.191917,2026-10-29,129.000,3.245000e+08,隆利转债,300752.XSHE,41.32
2,113016,-0.045631,15.70,6.609000e+08,4.298527e+09,121.206878,2023-11-06,125.770,1.500000e+09,小康转债,601127.XSHG,20.69
3,113557,0.131056,10.11,5.999400e+08,9.951769e+06,122.295625,2025-12-19,109.190,6.000000e+08,森特转债,603098.XSHG,9.76
4,123049,0.028049,7.48,9.170566e+08,5.408421e+07,122.514914,2026-04-13,119.710,9.172400e+08,维尔转债,300190.XSHE,8.71
5,128066,0.091261,9.67,4.798080e+08,3.334289e+06,122.879142,2025-04-17,113.753,4.800000e+08,亚泰转债,002811.XSHE,10.08
6,113562,0.014920,82.73,5.320920e+08,1.266005e+07,123.042042,2025-01-02,121.550,8.700000e+08,璞泰转债,603659.XSHG,99.08
7,113593,-0.024143,21.32,4.000000e+08,4.264842e+07,123.595659,2026-07-20,126.010,4.000000e+08,沪工转债,603131.XSHG,27.53
8,113549,0.136619,8.88,8.799120e+08,3.270785e+06,124.251944,2025-11-15,110.590,8.800000e+08,白电转债,603861.XSHG,8.64
9,113567,0.079351,11.46,2.099580e+08,3.727177e+06,124.535057,2026-03-04,116.600,2.100000e+08,君禾转债,603617.XSHG,12.38


In [7]:
# 采用pb>1.3筛选
bond_list_b = DLowStrategy(bond_list, base_date).get_support_bonds(filter_pb=True, filter_pb_pe_quantile=False)
pd.DataFrame(bond_list_b)

,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,price,raise_fund_volume,short_name,stock_code,stock_pb,stock_pb_quantile,stock_pe,stock_pe_quantile,stock_price,stock_roe
0,113601,0.078055,16.98,5.433100e+08,1.363249e+07,118.785515,2026-08-21,110.980,5.433100e+08,塞力转债,603716.XSHG,2.3551,0.405120,71.3621,0.880290,17.48,0.033002
1,123074,-0.078081,29.53,3.245000e+08,4.184119e+09,121.191917,2026-10-29,129.000,3.245000e+08,隆利转债,300752.XSHE,5.7128,0.859273,81.2925,1.000000,41.32,0.070275
2,113557,0.131056,10.11,5.999400e+08,9.951769e+06,122.295625,2025-12-19,109.190,6.000000e+08,森特转债,603098.XSHG,2.2600,0.117040,27.5624,0.481102,9.76,0.081996
3,123049,0.028049,7.48,9.170566e+08,5.408421e+07,122.514914,2026-04-13,119.710,9.172400e+08,维尔转债,300190.XSHE,1.6155,0.477140,17.5683,0.067975,8.71,0.091955
4,128066,0.091261,9.67,4.798080e+08,3.334289e+06,122.879142,2025-04-17,113.753,4.800000e+08,亚泰转债,002811.XSHE,1.8468,0.158171,37.3977,0.655950,10.08,0.049383
5,113562,0.014920,82.73,5.320920e+08,1.266005e+07,123.042042,2025-01-02,121.550,8.700000e+08,璞泰转债,603659.XSHG,11.8990,0.810465,71.3108,0.880860,99.08,0.166861
6,113593,-0.024143,21.32,4.000000e+08,4.264842e+07,123.595659,2026-07-20,126.010,4.000000e+08,沪工转债,603131.XSHG,7.0986,0.686832,81.1397,0.776851,27.53,0.087486
7,113549,0.136619,8.88,8.799120e+08,3.270785e+06,124.251944,2025-11-15,110.590,8.800000e+08,白电转债,603861.XSHG,1.6295,0.109554,101.7850,1.000000,8.64,0.016009
8,113567,0.079351,11.46,2.099580e+08,3.727177e+06,124.535057,2026-03-04,116.600,2.100000e+08,君禾转债,603617.XSHG,3.8314,0.414803,27.5893,0.181989,12.38,0.138873
9,113541,0.057833,11.54,2.745270e+08,1.329818e+06,125.133333,2025-07-23,119.350,3.300000e+08,荣晟转债,603165.XSHG,2.1461,0.061893,13.6154,0.414076,13.02,0.157623


In [8]:
# 采用pe, pb百分位筛选
bond_list_c = DLowStrategy(bond_list, base_date).get_support_bonds(filter_pb=True, filter_pb_pe_quantile=True)
pd.DataFrame(bond_list_c)


,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,price,raise_fund_volume,short_name,stock_code,stock_pb,stock_pb_quantile,stock_pe,stock_pe_quantile,stock_price,stock_roe
0,113557,0.131056,10.11,5.999400e+08,9951769.00,122.295625,2025-12-19,109.190,6.000000e+08,森特转债,603098.XSHG,2.2600,0.117040,27.5624,0.481102,9.76,0.081996
1,123049,0.028049,7.48,9.170566e+08,54084214.54,122.514914,2026-04-13,119.710,9.172400e+08,维尔转债,300190.XSHE,1.6155,0.477140,17.5683,0.067975,8.71,0.091955
2,128066,0.091261,9.67,4.798080e+08,3334288.56,122.879142,2025-04-17,113.753,4.800000e+08,亚泰转债,002811.XSHE,1.8468,0.158171,37.3977,0.655950,10.08,0.049383
3,113593,-0.024143,21.32,4.000000e+08,42648415.00,123.595659,2026-07-20,126.010,4.000000e+08,沪工转债,603131.XSHG,7.0986,0.686832,81.1397,0.776851,27.53,0.087486
4,113567,0.079351,11.46,2.099580e+08,3727177.00,124.535057,2026-03-04,116.600,2.100000e+08,君禾转债,603617.XSHG,3.8314,0.414803,27.5893,0.181989,12.38,0.138873
5,113541,0.057833,11.54,2.745270e+08,1329818.00,125.133333,2025-07-23,119.350,3.300000e+08,荣晟转债,603165.XSHG,2.1461,0.061893,13.6154,0.414076,13.02,0.157623
6,128105,0.101380,8.11,7.999200e+08,10383638.82,125.843988,2026-04-09,115.706,8.000000e+08,长集转债,002616.XSHE,2.5126,0.250076,17.3312,0.001218,8.52,0.144976
7,128116,0.147930,29.82,6.500000e+08,7556497.42,125.852046,2026-06-29,111.059,6.500000e+08,瑞达转债,002961.XSHE,7.3005,0.271975,80.1673,0.098298,28.85,0.091066
8,123063,0.070633,4.94,6.380000e+08,7324441.32,126.263273,2026-07-28,119.200,6.380000e+08,大禹转债,300021.XSHE,3.0382,0.325924,46.7707,0.394864,5.50,0.064959
9,128039,0.090523,5.81,2.025540e+08,20665206.03,126.363306,2024-06-08,117.311,6.200000e+08,三力转债,002224.XSHE,1.9491,0.059734,29.2599,0.124075,6.25,0.066613


In [9]:
# 采用价格过滤
bond_list_d = DLowStrategy(bond_list, base_date).get_support_bonds(filter_pb=True, filter_pb_pe_quantile=True, filter_price=True)
pd.DataFrame(bond_list_d)

""


In [10]:
bond_list_a_text = pd.DataFrame(bond_list_a).to_html()
bond_list_b_text = pd.DataFrame(bond_list_b).to_html()
bond_list_c_text = pd.DataFrame(bond_list_c).to_html()


# 取得几个统计值：市场总量，小于110元转债总量，价格算术平均值，溢价率算术平均值
total_market, underrate_market, avg_price, avg_premium_ratio = index_bond.get_bonds_factors(bond_list)
total_market_text = "市场总量{}亿元, 小于110元转债总量{}亿元, 价格算术平均值{}, 溢价率算术平均值{}".format(
                                                                total_market/100000000,
                                                                underrate_market/100000000,
                                                                avg_price, avg_premium_ratio
                    )

split_text = '=================================================='
print(total_market_text)

send_message_text = "{}\r\n{}\r\n{}\r\n{}\r\n{}\r\n{}\r\n".format(
    bond_list_a_text, split_text, bond_list_b_text, bond_list_c_text, split_text, total_market_text)
#print(send_message_text)

市场总量4688.9588878819995亿元, 小于110元转债总量1839.9555522123亿元, 价格算术平均值131.15867601246106, 溢价率算术平均值0.21446246951219444
